# Building Agent Tools with FastMCP

> Computational Analysis of Social Complexity
>
> Fall 2025, Spencer Lyon

**Prerequisites**

- L.A2.01 (Function calling and tool use)
- L.A2.02 (Type-safe agents with PydanticAI)
- L.A2.03 (Agent evaluations)
- Basic understanding of client-server architecture

**Outcomes**

- Understand the Model Context Protocol (MCP) and its role in the AI ecosystem
- Create MCP servers using FastMCP to expose computational tools
- Integrate MCP servers with PydanticAI agents for distributed tool access
- Deploy and test MCP servers in multiple environments (local, HTTP, Claude Desktop)
- Apply MCP patterns to course domains: network analysis, game theory, and agent-based models

**References**

- [Model Context Protocol Specification](https://modelcontextprotocol.io/)
- [FastMCP Documentation](https://gofastmcp.com/)
- [FastMCP GitHub Repository](https://github.com/jlowin/fastmcp)
- [Anthropic MCP Announcement](https://www.anthropic.com/news/model-context-protocol)
- [PydanticAI Documentation](https://ai.pydantic.dev/)
- [NetworkX Documentation](https://networkx.org/)
- [Mesa Documentation](https://mesa.readthedocs.io/)

## From Embedded Tools to Distributed Tools

### Review: The Week A2 Pattern

Last week, we learned how to build AI agents with tools using PydanticAI.

The pattern looked like this:

```python
from pydantic_ai import Agent

agent = Agent('anthropic:claude-haiku-4-5')

@agent.tool
def calculate_degree_centrality(graph_id: str, node: int) -> float:
    """Calculate degree centrality for a node in a network."""
    # Implementation here
    return centrality_score
```

This works beautifully. The agent can call our tools, we get type safety, and everything lives in one Python process.

But there's a problem.

### The Reusability Problem

Suppose you've built an amazing network analysis toolkit:
- Calculate centrality measures
- Find communities
- Compute shortest paths
- Analyze network structure

You spent hours implementing it with proper validation, error handling, and optimization.

Now you want to use these tools in:
1. Your PydanticAI agent for research
2. A ChatGPT plugin for students
3. A Claude Desktop integration
4. An API for your web app
5. A Jupyter notebook assistant

**Problem**: You have to reimplement the tools for each platform.

- ChatGPT wants OpenAPI spec
- Claude wants their tool format
- Your API needs REST endpoints
- Each has different authentication, deployment, testing

You end up maintaining **5 different implementations** of the same tools.

This is the **reusability problem**.

### The Integration Challenge

It gets worse.

When you write tools with `@agent.tool`, they're **locked into PydanticAI**. 

What if:
- You want to use a different agent framework?
- You want to share tools with colleagues using different tech stacks?
- You want tools to work in Claude Desktop without rewriting them?
- You want a student to use your tools from their own application?

Each framework has its own way of defining tools. There's no common language.

This is the **integration challenge**.

### Enter: The Model Context Protocol

What if there was a universal standard for AI tools?

Like how USB-C works with any device, what if there was "USB-C for AI"?

**That's the Model Context Protocol (MCP)**.

The idea is simple:

1. **Write your tools once** as an MCP server
2. **Use them anywhere** with any MCP-compatible client

```
┌─────────────────────────────────────┐
│     Your MCP Server                 │
│  (Network Analysis Tools)           │
│                                     │
│  - calculate_centrality()           │
│  - find_communities()               │
│  - shortest_path()                  │
└──────────────┬──────────────────────┘
               │
               │  MCP Protocol
               │
    ┌──────────┴──────────┐
    │                     │
    ▼                     ▼
┌─────────┐          ┌─────────┐
│ Claude  │          │ ChatGPT │
│ Desktop │          │  Plugin │
└─────────┘          └─────────┘
    │                     │
    ▼                     ▼
┌─────────┐          ┌─────────┐
│Pydantic │          │  Your   │
│   AI    │          │   App   │
└─────────┘          └─────────┘
```

Write once, use everywhere.

### The Three MCP Primitives

MCP defines three types of capabilities servers can expose:

**1. Tools** (Functions)
- Actions the AI can perform
- Examples: `calculate_centrality()`, `find_nash_equilibrium()`, `run_simulation()`
- What we've been working with in Week A2

**2. Resources** (Data)
- Read-only access to information
- Examples: network datasets, simulation results, research papers
- Like files or database queries

**3. Prompts** (Templates)
- Reusable message templates
- Examples: "Analyze this network", "Explain this game theory result"
- Workflows that users can invoke

Today we'll focus primarily on **Tools**, with a brief look at Resources and Prompts.

### Why This Matters

Think back to our course themes:

**Network Science (Weeks 3-5)**:
- You built expertise in graph theory
- You know how to calculate centrality, find communities, analyze structure
- What if **any AI agent** could access your network analysis capabilities?

**Game Theory (Weeks 8-9)**:
- You can solve games, find Nash equilibria, analyze auctions
- What if researchers could ask an AI to "solve this game" and it **actually computes** the answer?

**Agent-Based Models (Weeks 6-7)**:
- You built simulations of complex systems
- What if an AI could **run your simulations** and **interpret the results**?

MCP makes this possible.

Let's build it.

## FastMCP Fundamentals

### What is FastMCP?

FastMCP is a Python framework for building production-ready MCP servers.

It's created by the same team behind Pydantic (which powers OpenAI SDK, Anthropic SDK, FastAPI, and countless production systems).

**Philosophy**: The "Pydantic way"
- Type-safe by default
- Automatic validation
- Minimal boilerplate
- Production-ready patterns

**Core Concepts**:

1. **Server Creation**: One line to create an MCP server
2. **Tool Decoration**: `@mcp.tool` decorator (just like `@agent.tool`!)
3. **Automatic Schema Generation**: From type hints and docstrings
4. **Multiple Transports**: stdio (local), HTTP (remote), Claude Desktop

Let's see it in action.

### Installation

First, install FastMCP:

In [1]:
# !pip install fastmcp pydantic-ai networkx quantecon mesa

### Your First MCP Server: Calculator

Let's start with the simplest possible MCP server - a calculator.

This demonstrates the core pattern we'll use throughout.

In [2]:
from fastmcp import FastMCP

# Create an MCP server
mcp = FastMCP("Calculator")

# Define a tool
@mcp.tool()
def add(a: float, b: float) -> float:
    """Add two numbers together."""
    return a + b

@mcp.tool()
def multiply(a: float, b: float) -> float:
    """Multiply two numbers together."""
    return a * b

print("✓ Calculator MCP server created with 2 tools")

✓ Calculator MCP server created with 2 tools


**That's it**.

We just created an MCP server with two tools.

Notice:
- No JSON schema writing (FastMCP generates from type hints)
- No manual validation (Pydantic handles it)
- No complex setup (one line: `FastMCP("Calculator")`)

The pattern is identical to what we learned with PydanticAI:
- Type hints for parameters
- Docstring for description
- Decorator to register the tool

### Transport Protocols

MCP servers can run in different modes:

**1. stdio (Standard Input/Output)**
- For local communication
- Client launches server as subprocess
- Communication via stdin/stdout
- Use case: Claude Desktop, local testing

**2. StreamableHTTP**
- For remote communication
- Server runs on HTTP
- Clients connect over network
- Use case: Production deployments, web apps

We'll use stdio for local development and HTTP for production patterns.

### Running an MCP Server

In production, you'd run the server like this:


In [3]:
%%file calculator_server.py

# calculator_server.py
from fastmcp import FastMCP

mcp = FastMCP("Calculator")

@mcp.tool()
def add(a: float, b: float) -> float:
    """Add two numbers together."""
    return a + b

# Run the server
if __name__ == "__main__":
    mcp.run()  # stdio by default

Overwriting calculator_server.py


Then from command line:
```bash
python calculator_server.py
```

For HTTP:
```python
mcp.run(transport="http", port=8000)
```


### Testing an MCP Server

We can test our server programmatically using the FastMCP Client:

In [4]:
from fastmcp import Client

async with Client("calculator_server.py") as client:
    tools = await client.list_tools()
    print("Available tools:", [t.name for t in tools])

    result = await client.call_tool("add", {"a": 5, "b": 3})
    print("5 + 3 =", result)

print("Note: Full client testing requires running server as separate process")
print("We'll see integration with PydanticAI shortly")

Available tools: ['add']
5 + 3 = CallToolResult(content=[TextContent(type='text', text='8.0', annotations=None, meta=None)], structured_content={'result': 8.0}, data=8.0, is_error=False)
Note: Full client testing requires running server as separate process
We'll see integration with PydanticAI shortly


### Key Insight

The calculator server we just built is **immediately usable** by:
- Claude Desktop (via `fastmcp install claude-desktop calculator_server.py`)
- Any PydanticAI agent (via FastMCP Client)
- Custom applications (via MCP protocol)

Write once, use everywhere.

Now let's build something more interesting.

## Building Course-Specific MCP Servers

### Network Analysis MCP Server

Remember Weeks 3-5? We studied network science:
- Graph theory fundamentals
- Centrality measures
- Community detection
- Network structure

Let's build an MCP server that makes these capabilities available to any AI agent.

We'll use **NetworkX**, the canonical Python library for network analysis.

#### State Management: The Challenge

Our network analysis server needs to **remember graphs** between tool calls.

When an agent says:
1. "Create a network with these edges"
2. "Calculate centrality for node 5"

The server needs to remember the graph from step 1.

**Important**: FastMCP's Context is **request-scoped** - each tool call gets a fresh context.

From the FastMCP documentation:
> "State set during one request will not be available in subsequent requests. For persistent data storage across requests, use external storage mechanisms like databases, files, or in-memory caches."

**Solution**: Use a **global dictionary** outside of Context. This simple cache persists across tool calls:

```python
cache: Dict[str, Any] = {}  # Global cache, lives for server lifetime
```

In production, you'd use Redis, a database, or file storage. For our purposes, a dict works perfectly.

In [5]:
%%file network_analysis_server.py

# network_analysis_server.py
from fastmcp import FastMCP
import networkx as nx
from typing import Dict, List, Tuple, Any


# Global cache for persistent state across tool calls
# MCP Context is request-scoped, so we need external storage
# A simple dict works perfectly for our purposes
cache: Dict[str, Any] = {}


# Create network analysis server
network_mcp = FastMCP("NetworkAnalysis")


@network_mcp.tool()
def create_network(
    graph_id: str,
    edges: List[Tuple[int, int]]
) -> Dict[str, Any]:
    """
    Create a network from an edge list and store it.

    Args:
        graph_id: Unique identifier for this graph (e.g., 'social_network', 'graph1')
        edges: List of edges as [source, target] pairs. Example: [[1,2], [2,3], [1,3]]

    Returns:
        Dictionary with graph statistics (num_nodes, num_edges, density)
    """
    # Create NetworkX graph
    G = nx.Graph()
    G.add_edges_from(edges)

    # Store in global cache (persists across tool calls!)
    cache[f"graph:{graph_id}"] = G

    return {
        "graph_id": graph_id,
        "num_nodes": G.number_of_nodes(),
        "num_edges": G.number_of_edges(),
        "density": round(nx.density(G), 4)
    }


@network_mcp.tool()
def calculate_degree_centrality(
    graph_id: str,
    node: int
) -> Dict[str, Any]:
    """
    Calculate degree centrality for a node.

    Degree centrality measures how many connections a node has.
    Higher values indicate more central/connected nodes.

    Args:
        graph_id: ID of the graph to analyze
        node: The node ID to calculate centrality for

    Returns:
        Dictionary with degree and normalized centrality value
    """
    # Retrieve graph from global cache
    G = cache.get(f"graph:{graph_id}")
    
    if G is None:
        return {"error": f"Graph '{graph_id}' not found. Create it first."}

    if node not in G:
        return {"error": f"Node {node} not in graph '{graph_id}'"}

    degree = G.degree(node)
    max_possible = G.number_of_nodes() - 1
    normalized = degree / max_possible if max_possible > 0 else 0

    return {
        "node": node,
        "degree": degree,
        "normalized_centrality": round(normalized, 4)
    }


@network_mcp.tool()
def calculate_betweenness(
    graph_id: str,
    node: int
) -> Dict[str, Any]:
    """
    Calculate betweenness centrality for a node.

    Betweenness measures how often a node lies on shortest paths between other nodes.
    High betweenness nodes are 'bridges' connecting different parts of the network.

    Args:
        graph_id: ID of the graph to analyze
        node: The node ID to calculate betweenness for

    Returns:
        Dictionary with betweenness centrality value
    """
    G = cache.get(f"graph:{graph_id}")
    
    if G is None:
        return {"error": f"Graph '{graph_id}' not found"}

    if node not in G:
        return {"error": f"Node {node} not in graph '{graph_id}'"}

    betweenness = nx.betweenness_centrality(G)

    return {
        "node": node,
        "betweenness_centrality": round(betweenness[node], 4)
    }


@network_mcp.tool()
def find_shortest_path(
    graph_id: str,
    source: int,
    target: int
) -> Dict[str, Any]:
    """
    Find shortest path between two nodes.

    Args:
        graph_id: ID of the graph to search
        source: Starting node ID
        target: Destination node ID

    Returns:
        Dictionary with path and length, or error if no path exists
    """
    G = cache.get(f"graph:{graph_id}")
    
    if G is None:
        return {"error": f"Graph '{graph_id}' not found"}

    try:
        path = nx.shortest_path(G, source, target)
        return {
            "found": True,
            "path": path,
            "length": len(path) - 1
        }
    except nx.NetworkXNoPath:
        return {
            "found": False,
            "message": f"No path exists between {source} and {target}"
        }
    except nx.NodeNotFound:
        return {
            "found": False,
            "message": f"One or both nodes not in graph"
        }


# Run the server
if __name__ == "__main__":
    network_mcp.run()  # stdio by default

Overwriting network_analysis_server.py


### What We Just Built

This MCP server exposes network analysis capabilities from Weeks 3-5.

**Key Features**:

1. **State Management**
   - Global `cache` dict stores graphs between calls
   - Agent can create a graph, then analyze it
   - Multiple graphs can coexist (different `graph_id`s)
   - Pattern: `cache[f"graph:{graph_id}"] = G`

2. **Type Safety**
   - All inputs validated automatically
   - Type hints ensure correct data types
   - Clear error messages for invalid inputs

3. **Domain Expertise Encoded**
   - Docstrings explain **when** to use each tool
   - Return values are structured and self-documenting
   - Error handling guides the agent

4. **Natural Language Interface**
   - Agent can say "find the shortest path from 1 to 5"
   - Server translates to `nx.shortest_path(G, 1, 5)`
   - Result returned in natural language-friendly format

**This is powerful**: Any AI agent can now perform network analysis by calling these tools.

No need to understand NetworkX. No need to write code. Just natural language.

### Testing the Network Analysis Server

Since we wrote the server to a file using `%%file`, we can now test it using the FastMCP Client. 

Let's explore how to interact with our network analysis server programmatically:

In [6]:
from fastmcp import Client

async with Client("network_analysis_server.py") as client:
    # 1. Discover available tools
    tools = await client.list_tools()
    print("Available tools:")
    for tool in tools:
        print(f"  - {tool.name}")
    
    print("\n" + "="*50 + "\n")
    
    # 2. Create a social network (small-world structure)
    print("Creating a social network...")
    result = await client.call_tool(
        "create_network",
        {
            "graph_id": "social_network",
            "edges": [
                [1, 2], [1, 3], [2, 3],  # Triangle cluster
                [3, 4], [4, 5], [5, 6], [6, 4],  # Another cluster
                [3, 7], [7, 8], [8, 9], [9, 7]   # Third cluster
            ]
        }
    )
    print(f"Network created: {result.data}")
    
    print("\n" + "="*50 + "\n")
    
    # 3. Analyze centrality - who's most important?
    print("Calculating degree centrality for node 3 (bridge node)...")
    result = await client.call_tool(
        "calculate_degree_centrality",
        {"graph_id": "social_network", "node": 3}
    )
    print(f"Node 3 centrality: {result.data}")
    
    print("\nCalculating degree centrality for node 1 (peripheral node)...")
    result = await client.call_tool(
        "calculate_degree_centrality",
        {"graph_id": "social_network", "node": 1}
    )
    print(f"Node 1 centrality: {result.data}")
    
    print("\n" + "="*50 + "\n")
    
    # 4. Calculate betweenness - who controls information flow?
    print("Calculating betweenness centrality for node 3...")
    result = await client.call_tool(
        "calculate_betweenness",
        {"graph_id": "social_network", "node": 3}
    )
    print(f"Node 3 betweenness: {result.data}")
    
    print("\n" + "="*50 + "\n")
    
    # 5. Find shortest paths
    print("Finding shortest path from node 1 to node 9...")
    result = await client.call_tool(
        "find_shortest_path",
        {"graph_id": "social_network", "source": 1, "target": 9}
    )
    print(f"Path result: {result.data}")
    
    print("\nFinding shortest path from node 2 to node 6...")
    result = await client.call_tool(
        "find_shortest_path",
        {"graph_id": "social_network", "source": 2, "target": 6}
    )
    print(f"Path result: {result.data}")

Available tools:
  - create_network
  - calculate_degree_centrality
  - calculate_betweenness
  - find_shortest_path


Creating a social network...
Network created: {'graph_id': 'social_network', 'num_nodes': 9, 'num_edges': 11, 'density': 0.3056}


Calculating degree centrality for node 3 (bridge node)...
Node 3 centrality: {'node': 3, 'degree': 4, 'normalized_centrality': 0.5}

Calculating degree centrality for node 1 (peripheral node)...
Node 1 centrality: {'node': 1, 'degree': 2, 'normalized_centrality': 0.25}


Calculating betweenness centrality for node 3...
Node 3 betweenness: {'node': 3, 'betweenness_centrality': 0.75}


Finding shortest path from node 1 to node 9...
Path result: {'found': True, 'path': [1, 3, 7, 9], 'length': 3}

Finding shortest path from node 2 to node 6...
Path result: {'found': True, 'path': [2, 3, 4, 6], 'length': 3}


### Interpreting the Results

Notice what just happened:

1. **Tool Discovery**: The client automatically discovered all 4 tools available on the server
2. **State Persistence**: We created a network, then analyzed it in subsequent calls - the server remembered the graph
3. **Structured Results**: Each tool returned a dictionary with clear, semantic field names
4. **Network Insights**: 
   - Node 3 has the highest degree (4 connections) and acts as a bridge between clusters
   - Node 3 also has high betweenness centrality - it controls information flow
   - The shortest path from node 1 to node 9 must go through node 3 (the bridge)

**This is the power of MCP**: We're performing sophisticated network analysis through a simple client API. The same server could be called from Claude Desktop, a web application, or any other MCP-compatible client.

Let's see one more example - handling errors gracefully:

In [7]:
# Demonstrating error handling
async with Client("network_analysis_server.py") as client:
    # Try to analyze a non-existent graph
    print("Attempting to analyze a non-existent graph...")
    result = await client.call_tool(
        "calculate_degree_centrality",
        {"graph_id": "does_not_exist", "node": 1}
    )
    print(f"Result: {result.data}")
    
    print("\n" + "="*50 + "\n")
    
    # Create a graph, then try to access a non-existent node
    print("Creating a small graph...")
    await client.call_tool(
        "create_network",
        {"graph_id": "test_graph", "edges": [[1, 2], [2, 3]]}
    )
    
    print("Attempting to analyze non-existent node 999...")
    result = await client.call_tool(
        "calculate_degree_centrality",
        {"graph_id": "test_graph", "node": 999}
    )
    print(f"Result: {result.data}")

print("\nThe server provides clear error messages that guide the AI agent")

Attempting to analyze a non-existent graph...
Result: {'error': "Graph 'does_not_exist' not found. Create it first."}


Creating a small graph...
Attempting to analyze non-existent node 999...
Result: {'error': "Node 999 not in graph 'test_graph'"}

The server provides clear error messages that guide the AI agent


### Game Theory MCP Server

Let's apply the same pattern to game theory (Weeks 8-9).

We'll use `quantecon.game_theory` for equilibrium computation:

In [8]:
%%file game_server.py

from fastmcp import FastMCP
import quantecon.game_theory as gt
import numpy as np
from typing import Dict, List, Any

game_mcp = FastMCP("GameTheory")

# Global cache for games (same pattern as network server)
game_cache: Dict[str, Any] = {}

@game_mcp.tool()
def create_game(
    game_id: str,
    payoff_matrix_p1: List[List[float]],
    payoff_matrix_p2: List[List[float]]
) -> Dict[str, Any]:
    """
    Create a two-player normal-form game.

    Args:
        game_id: Unique identifier for this game
        payoff_matrix_p1: Payoff matrix for Player 1 (rows = P1 strategies, cols = P2 strategies)
        payoff_matrix_p2: Payoff matrix for Player 2 (rows = P1 strategies, cols = P2 strategies)

    Returns:
        Game statistics and confirmation
    """
    # Convert to numpy arrays
    p1_payoffs = np.array(payoff_matrix_p1)
    p2_payoffs = np.array(payoff_matrix_p2)

    # Create game
    game = gt.NormalFormGame([p1_payoffs, p2_payoffs])

    # Store in global cache
    game_cache[game_id] = game

    return {
        "game_id": game_id,
        "num_players": 2,
        "p1_strategies": p1_payoffs.shape[0],
        "p2_strategies": p1_payoffs.shape[1],
        "message": f"Game '{game_id}' created successfully"
    }

@game_mcp.tool()
def find_pure_nash_equilibria(
    game_id: str
) -> Dict[str, Any]:
    """
    Find all pure strategy Nash equilibria in the game.

    A Nash equilibrium is a strategy profile where no player can improve
    by unilaterally changing their strategy.

    Args:
        game_id: ID of the game to analyze

    Returns:
        List of Nash equilibria (strategy profiles) and their payoffs
    """
    game = game_cache.get(game_id)
    
    if game is None:
        return {"error": f"Game '{game_id}' not found"}

    equilibria = game.pure_nash_brute()

    results = []
    for eq in equilibria:
        # eq is a tuple of strategy indices
        payoffs = [game.players[i].payoff_array[eq] for i in range(len(game.players))]
        results.append({
            "strategies": eq,
            "payoffs": [float(p) for p in payoffs]
        })

    return {
        "game_id": game_id,
        "num_equilibria": len(results),
        "equilibria": results
    }

@game_mcp.tool()
def check_not_dominated(
    game_id: str,
    player: int,
    strategy: int
) -> Dict[str, Any]:
    """
    Check if a strategy is not dominated for a player.

    A dominated strategy is one that always yields lower payoff than some other strategy.
    A not-dominated strategy may be part of a Nash equilibrium.

    Args:
        game_id: ID of the game
        player: Player number (0 or 1)
        strategy: Strategy index to check

    Returns:
        Whether the strategy is dominated and analysis
    """
    game = game_cache.get(game_id)
    
    if game is None:
        return {"error": f"Game '{game_id}' not found"}

    if player not in [0, 1]:
        return {"error": "Player must be 0 or 1"}

    is_dominated = game.players[player].is_dominated(strategy)

    return {
        "game_id": game_id,
        "player": player,
        "strategy": strategy,
        "is_dominated": bool(is_dominated),
        "is_not_dominated": not is_dominated,
        "explanation": "This strategy is dominated by another strategy" if is_dominated else "This strategy is not dominated and may be part of a Nash equilibrium"
    }

print("✓ Game Theory MCP server created with 3 tools")

Overwriting game_server.py


### Agent-Based Model Controller

Finally, let's create an MCP server for running agent-based models (Weeks 6-7).

We'll use **Mesa**, the Python framework for ABMs:

In [9]:
%%file abm_server.py

from fastmcp import FastMCP
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import SingleGrid
from mesa.datacollection import DataCollector
from typing import Dict, List, Any
import random

abm_mcp = FastMCP("AgentBasedModels")

# Global cache for ABM models
abm_cache: Dict[str, Any] = {}

# Define a simple Schelling segregation agent
class SchellingAgent(Agent):
    def __init__(self, unique_id, model, agent_type):
        super().__init__(unique_id, model)
        self.type = agent_type

    def step(self):
        # Find neighbors
        neighbors = self.model.grid.get_neighbors(
            self.pos, moore=True, include_center=False
        )

        # Count similar neighbors
        similar = sum(1 for n in neighbors if n.type == self.type)
        total = len(neighbors)

        # Move if unhappy (less than threshold% similar)
        if total > 0 and (similar / total) < self.model.homophily:
            self.model.grid.move_to_empty(self)

class SchellingModel(Model):
    def __init__(self, width=20, height=20, density=0.8, minority_pc=0.2, homophily=3):
        super().__init__()
        self.width = width
        self.height = height
        self.density = density
        self.minority_pc = minority_pc
        self.homophily = homophily / 8  # Convert to ratio

        self.schedule = RandomActivation(self)
        self.grid = SingleGrid(width, height, torus=True)

        # Create agents
        n_agents = int(width * height * density)
        for i in range(n_agents):
            agent_type = 1 if random.random() < minority_pc else 0
            agent = SchellingAgent(i, self, agent_type)
            self.schedule.add(agent)

            # Place randomly
            x = random.randrange(width)
            y = random.randrange(height)
            self.grid.position_agent(agent, (x, y))

        # Data collector
        self.datacollector = DataCollector(
            model_reporters={
                "segregation": lambda m: self.measure_segregation(m)
            }
        )

    @staticmethod
    def measure_segregation(model):
        """Calculate segregation metric (0-1, higher = more segregated)."""
        similar_neighbors = 0
        total_neighbors = 0

        for agent in model.schedule.agents:
            neighbors = model.grid.get_neighbors(
                agent.pos, moore=True, include_center=False
            )
            if neighbors:
                similar = sum(1 for n in neighbors if n.type == agent.type)
                similar_neighbors += similar
                total_neighbors += len(neighbors)

        return similar_neighbors / total_neighbors if total_neighbors > 0 else 0

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

@abm_mcp.tool()
def create_schelling_model(
    model_id: str,
    width: int = 20,
    height: int = 20,
    density: float = 0.8,
    minority_percent: float = 0.2,
    homophily: int = 3
) -> Dict[str, Any]:
    """
    Create a Schelling segregation model.

    The Schelling model demonstrates how mild preferences for similar neighbors
    can lead to high levels of segregation.

    Args:
        model_id: Unique identifier for this model
        width: Grid width (default 20)
        height: Grid height (default 20)
        density: Fraction of cells occupied (0-1, default 0.8)
        minority_percent: Fraction of agents that are minority type (0-1, default 0.2)
        homophily: Number of similar neighbors desired (out of 8, default 3)

    Returns:
        Model configuration and initial state
    """
    model = SchellingModel(width, height, density, minority_percent, homophily)

    # Store in global cache
    abm_cache[model_id] = model

    return {
        "model_id": model_id,
        "width": width,
        "height": height,
        "num_agents": len(model.schedule.agents),
        "initial_segregation": round(model.measure_segregation(model), 3)
    }

@abm_mcp.tool()
def step_model(
    model_id: str,
    num_steps: int = 1
) -> Dict[str, Any]:
    """
    Run the model for a specified number of steps.

    Args:
        model_id: ID of the model to step
        num_steps: Number of steps to run (default 1)

    Returns:
        Segregation metrics after stepping
    """
    model = abm_cache.get(model_id)
    
    if model is None:
        return {"error": f"Model '{model_id}' not found"}

    for _ in range(num_steps):
        model.step()

    df = model.datacollector.get_model_vars_dataframe()

    return {
        "model_id": model_id,
        "steps_completed": num_steps,
        "total_steps": len(df),
        "current_segregation": round(df['segregation'].iloc[-1], 3),
        "initial_segregation": round(df['segregation'].iloc[0], 3)
    }

@abm_mcp.tool()
def get_segregation_metric(
    model_id: str
) -> Dict[str, Any]:
    """
    Get current segregation level in the model.

    Args:
        model_id: ID of the model to query

    Returns:
        Current and historical segregation metrics
    """
    model = abm_cache.get(model_id)
    
    if model is None:
        return {"error": f"Model '{model_id}' not found"}

    current_seg = model.measure_segregation(model)

    df = model.datacollector.get_model_vars_dataframe()

    return {
        "model_id": model_id,
        "current_segregation": round(current_seg, 3),
        "mean_segregation": round(df['segregation'].mean(), 3),
        "max_segregation": round(df['segregation'].max(), 3),
        "total_steps": len(df)
    }

print("✓ ABM MCP server created with 3 tools")

Overwriting abm_server.py


### What We've Accomplished

We now have **three MCP servers** that expose capabilities from our course:

1. **Network Analysis** (Weeks 3-5)
   - Create networks, calculate centrality, find paths
   - Any AI can now do network science

2. **Game Theory** (Weeks 8-9)
   - Create games, find equilibria, check dominance
   - Any AI can now solve games

3. **Agent-Based Models** (Weeks 6-7)
   - Create Schelling model, run simulations, measure emergence
   - Any AI can now run computational experiments

**Key Patterns**:

- **Global Cache for State**: `cache: Dict[str, Any] = {}` stores data between calls
- **No Context Dependency**: Tools access global cache, not request-scoped Context
- **Type Safety**: All inputs validated automatically
- **Clear Errors**: Guide the AI when something goes wrong
- **Structured Returns**: Dictionaries with semantic field names
- **Documentation**: Docstrings explain **what** and **when**

These servers are production-ready and reusable across any MCP-compatible client.

## Resources and Prompts

### MCP Resources: Read-Only Data

In addition to **Tools** (functions), MCP servers can expose **Resources** (data).

Resources are for read-only access to information:
- Datasets
- Simulation results
- Documentation
- Configuration files

**Example Use Cases**:
- Network adjacency matrices
- Historical game theory results
- ABM simulation outputs
- Research paper abstracts

Let's add a resource to our network analysis server:

In [10]:
from fastmcp import FastMCP, Context

# Create a new server with resources
network_resources_mcp = FastMCP("NetworkResources")

@network_resources_mcp.resource("network://{graph_id}/adjacency")
def get_adjacency_matrix(
    ctx: Context,
    graph_id: str
) -> str:
    """
    Get the adjacency matrix representation of a network.

    Resources use URI templates (RFC 6570).
    Clients can request: network://my_graph/adjacency
    """
    if not hasattr(ctx, 'graphs') or graph_id not in ctx.graphs:
        return f"Error: Graph '{graph_id}' not found"

    G = ctx.graphs[graph_id]
    adj_matrix = nx.adjacency_matrix(G).todense()

    return str(adj_matrix)

@network_resources_mcp.resource("network://{graph_id}/summary")
def get_network_summary(
    ctx: Context,
    graph_id: str
) -> str:
    """
    Get a text summary of network properties.
    """
    if not hasattr(ctx, 'graphs') or graph_id not in ctx.graphs:
        return f"Error: Graph '{graph_id}' not found"

    G = ctx.graphs[graph_id]

    summary = f"""
Network Summary: {graph_id}
==================
Nodes: {G.number_of_nodes()}
Edges: {G.number_of_edges()}
Density: {nx.density(G):.4f}
Average Degree: {sum(dict(G.degree()).values()) / G.number_of_nodes():.2f}
Is Connected: {nx.is_connected(G)}
"""
    return summary

print("✓ Network resources server created")

✓ Network resources server created


### Integration Architecture

Now comes the magic: connecting our MCP servers to PydanticAI agents.

PydanticAI has **native MCP support** built in. MCP servers are treated as **toolsets** that can be directly registered with agents.

The architecture looks like this:

```
┌─────────────────────────────────────┐
│      PydanticAI Agent               │
│   (Your AI Application)             │
└──────────────┬──────────────────────┘
               │
               │  toolsets=[server]
               ▼
┌─────────────────────────────────────┐
│      MCP Server Connection          │
│   (MCPServerStdio, FastMCPToolset)  │
└──────────────┬──────────────────────┘
               │
               │  MCP Protocol
               ▼
┌─────────────────────────────────────┐
│      MCP Servers                    │
│   - Network Analysis                │
│   - Game Theory                     │
│   - ABM Controller                  │
└─────────────────────────────────────┘
```

PydanticAI provides multiple ways to connect:
1. **MCPServerStdio** - Run server as subprocess (stdio transport)
2. **MCPServerStreamableHTTP** - Connect to HTTP server
3. **FastMCPToolset** - High-level integration via FastMCP Client

### Using MCPServerStdio with PydanticAI

The simplest integration - run your MCP server as a subprocess:

```python
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerStdio

# Create connection to our network analysis server
server = MCPServerStdio(
    'python', 
    args=['network_analysis_server.py']
)

# Create agent with MCP server as toolset
agent = Agent('anthropic:claude-sonnet-4-5', toolsets=[server])

async def main():
    # Agent automatically has access to all MCP tools!
    result = await agent.run(
        'Create a triangle network with nodes 1,2,3 and edges between all pairs. '
        'Then calculate the degree centrality of node 1.'
    )
    print(result.output)

# Run with: asyncio.run(main())
```

**What's happening here?**
1. `MCPServerStdio` launches the server as a subprocess
2. The agent discovers all tools from the server automatically
3. When the agent needs network analysis, it calls the MCP tools
4. Communication happens over stdin/stdout


### Using FastMCPToolset (Simpler Integration)

FastMCP provides an even simpler integration pattern:

In [11]:
from pydantic_ai import Agent
from pydantic_ai.toolsets.fastmcp import FastMCPToolset
from dotenv import load_dotenv

load_dotenv()

# Connect to MCP server (many options!)
# toolset = FastMCPToolset('network_analysis_server.py')

# Or directly from a FastMCP server object (no network overhead!)
from network_analysis_server import network_mcp
toolset = FastMCPToolset(network_mcp)

agent = Agent('anthropic:claude-haiku-4-5', toolsets=[toolset])

async def main():
    prompt = """
    I have a social network with the following friendships (edges):
    - Person 1 is friends with persons 2, 3, and 4
    - Person 2 is friends with persons 1 and 3
    - Person 3 is friends with persons 1, 2, and 4
    - Person 4 is friends with persons 1 and 3
    - Person 5 is friends with nobody

    Create this network and analyze its structure.

    think carefully, proceed step by step.
    """
    result = await agent.run(prompt)
    print(result.output)

await main()

## Network Analysis Results

### **Overall Network Statistics:**
- **Number of Nodes:** 4 (connected nodes) + 1 isolated node (Person 5)
- **Number of Edges:** 5
- **Network Density:** 0.833 (83.3%)
  - This is quite high, indicating a very well-connected group. The maximum possible would be 1.0 (complete graph).

### **Degree Centrality (How Connected Each Person Is):**

| Person | Degree | Normalized Centrality | Interpretation |
|--------|--------|----------------------|-----------------|
| **1** | 3 | 1.0 | **Hub** - Most connected |
| **2** | 2 | 0.667 | Moderately connected |
| **3** | 3 | 1.0 | **Hub** - Most connected |
| **4** | 2 | 0.667 | Moderately connected |
| **5** | 0 | 0.0 | Isolated - No friends |

### **Betweenness Centrality (Bridge Roles):**

| Person | Betweenness | Interpretation |
|--------|------------|-----------------|
| **1** | 0.167 | Acts as a bridge for some connections |
| **2** | 0.0 | Not a bridge (direct connections available) |
| **3** | 0.167 | Acts

**FastMCPToolset can connect to:**
- Python scripts: `FastMCPToolset('my_server.py')`
- HTTP URLs: `FastMCPToolset('http://localhost:8000/mcp')`
- FastMCP objects: `FastMCPToolset(network_mcp)`
- JSON config: `FastMCPToolset({'mcpServers': {...}})`

### Direct FastMCP Server Integration

If your MCP server is in the same codebase, you can skip the network entirely:

```python
from fastmcp import FastMCP
from pydantic_ai import Agent
from pydantic_ai.toolsets.fastmcp import FastMCPToolset

# Define server (same as before)
network_mcp = FastMCP("NetworkAnalysis")

@network_mcp.tool()
def create_network(graph_id: str, edges: list) -> dict:
    # ... implementation
    pass

# Create toolset directly from server object
toolset = FastMCPToolset(network_mcp)

# Agent uses tools without any network overhead
agent = Agent('anthropic:claude-sonnet-4-5', toolsets=[toolset])
```

This is **zero-overhead** - the agent calls the tools directly.

### Multi-Server Agents

The real power: connecting to **multiple MCP servers**:

```python
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerStdio

# Connect to multiple servers
network_server = MCPServerStdio('python', args=['network_analysis_server.py'])
game_server = MCPServerStdio('python', args=['game_theory_server.py'])
abm_server = MCPServerStdio('python', args=['abm_server.py'])

# Agent has tools from all three domains!
agent = Agent(
    'anthropic:claude-sonnet-4-5', 
    toolsets=[network_server, game_server, abm_server]
)

async def main():
    # Agent orchestrates across servers
    result = await agent.run(
        'Create a network, model it as a coordination game, '
        'and run an ABM simulation of agent behavior'
    )
    print(result.output)
```

Using tool prefixes to avoid naming conflicts:

```python
from pydantic_ai.mcp import MCPServerStdio

network_server = MCPServerStdio(
    'python', 
    args=['network_server.py'],
    tool_prefix='network'  # Tools prefixed with 'network_'
)

game_server = MCPServerStdio(
    'python', 
    args=['game_server.py'],
    tool_prefix='game'  # Tools prefixed with 'game_'
)
```

The agent **orchestrates** across servers:
- "Analyze the social network" → network server
- "Model this as a game" → game theory server
- "Run a simulation" → ABM server

**This is powerful**: compose capabilities like Lego blocks.

### Live Example: PydanticAI Agent with MCP Tools

Let's see this in action. We'll create a PydanticAI agent that uses our network analysis MCP server:

### Local Development: stdio Transport

For development and testing, use stdio transport.

**File: `network_server.py`**
```python
from fastmcp import FastMCP
# ... define tools ...

if __name__ == "__main__":
    mcp.run()  # stdio by default
```

**Run:**
```bash
python network_server.py
```

Server communicates via stdin/stdout. Perfect for local testing.

### Claude Desktop Integration

FastMCP has **built-in Claude Desktop support**.

Install your server:
```bash
fastmcp install claude-desktop network_server.py
```

This:
1. Adds server to Claude Desktop config
2. Makes tools available in Claude Desktop app
3. Users can interact naturally: "Calculate centrality for node 5"

**This is incredibly powerful**:
- Your research tools → available in Claude Desktop
- Students can use them without coding
- Natural language interface to computational methods

### HTTP Deployment (Streamable HTTP)

For production, deploy as HTTP server using the **Streamable HTTP** transport:

```python
# network_server.py
from fastmcp import FastMCP
# ... define tools ...

if __name__ == "__main__":
    mcp.run(transport='streamable-http', port=8000)
```

**Run:**
```bash
python network_server.py
```

Server runs on `http://localhost:8000/mcp`.

**Connect from PydanticAI using MCPServerStreamableHTTP:**
```python
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerStreamableHTTP

server = MCPServerStreamableHTTP('http://localhost:8000/mcp')
agent = Agent('anthropic:claude-sonnet-4-5', toolsets=[server])

async def main():
    result = await agent.run('Analyze the network')
    print(result.output)
```

**Or using FastMCPToolset:**
```python
from pydantic_ai import Agent
from pydantic_ai.toolsets.fastmcp import FastMCPToolset

toolset = FastMCPToolset('http://localhost:8000/mcp')
agent = Agent('anthropic:claude-sonnet-4-5', toolsets=[toolset])
```

**Deploy to production**:
- Run on cloud server (AWS, GCP, Azure)
- Use Docker for containerization
- Add authentication and rate limiting
- Scale horizontally as needed

### Loading MCP Servers from Configuration

You can load multiple servers from a JSON configuration file:

```json
{
  "mcpServers": {
    "network-analysis": {
      "command": "python",
      "args": ["network_analysis_server.py"]
    },
    "game-theory": {
      "url": "http://localhost:8001/mcp"
    },
    "abm-controller": {
      "command": "python",
      "args": ["abm_server.py"]
    }
  }
}
```

Load with PydanticAI:
```python
from pydantic_ai import Agent
from pydantic_ai.mcp import load_mcp_servers

servers = load_mcp_servers('mcp_config.json')
agent = Agent('anthropic:claude-sonnet-4-5', toolsets=servers)
```

Or with FastMCPToolset:
```python
from pydantic_ai.toolsets.fastmcp import FastMCPToolset

toolset = FastMCPToolset({'mcpServers': {...}})
agent = Agent('anthropic:claude-sonnet-4-5', toolsets=[toolset])
```

**Benefits**:
- Configure servers externally without code changes
- Support for environment variables (`${VAR:-default}`)
- Easy to manage multiple servers

### FastMCP Cloud

FastMCP offers hosted deployment:

1. Push your server to GitHub
2. Connect repo to FastMCP Cloud
3. Automatic deployment on push
4. Get a public URL for your server

**Benefits**:
- Zero infrastructure management
- Automatic scaling
- Built-in monitoring
- Easy sharing with collaborators

Perfect for research projects and teaching.

### Testing MCP Servers

Use pytest for testing:

```python
# test_network_server.py
import pytest
from pydantic_ai import Agent
from pydantic_ai.mcp import MCPServerStdio

@pytest.mark.asyncio
async def test_network_analysis_agent():
    server = MCPServerStdio('python', args=['network_analysis_server.py'])
    agent = Agent('anthropic:claude-sonnet-4-5', toolsets=[server])
    
    async with agent:
        result = await agent.run('Create a triangle network and count the edges')
        # Verify the agent used the tools correctly
        assert '3' in result.data  # Triangle has 3 edges

@pytest.mark.asyncio
async def test_direct_mcp_client():
    from fastmcp import Client
    
    async with Client("network_analysis_server.py") as client:
        result = await client.call_tool(
            "create_network",
            {"graph_id": "test", "edges": [[1, 2], [2, 3]]}
        )
        assert result.data["num_nodes"] == 3
        assert result.data["num_edges"] == 2
```

**Run tests:**
```bash
pytest test_network_server.py
```

### Integration with Evaluation Frameworks

Remember Week A2.03 (Evaluations)?

We can evaluate agents that use MCP servers:

```python
from pydantic_evals import Case, Dataset, evaluate

# Define test cases
network_eval_dataset = Dataset(
    cases=[
        Case(
            input="Create a triangle network and find the most central node",
            expected_output={"node": 1}  # All nodes equally central
        ),
        Case(
            input="Create a star network and identify the center",
            expected_output={"node": 1}  # Center has highest centrality
        ),
    ]
)

# Evaluate agent with MCP tools
results = evaluate(
    task=network_analysis_agent,
    dataset=network_eval_dataset,
    evaluators=[...]
)
```

This ensures your MCP servers work correctly with AI agents.

### Alternative: FastMCPToolset Integration

For an even simpler integration, use `FastMCPToolset`. This is especially powerful when the server is in the same codebase:

In [13]:
# Alternative: FastMCPToolset for simpler integration
from pydantic_ai import Agent
from pydantic_ai.toolsets.fastmcp import FastMCPToolset

# Option 1: Connect to Python script (runs as subprocess)
toolset = FastMCPToolset('network_analysis_server.py')

# Option 2: Connect directly to FastMCP server object (zero network overhead!)
# This is great when server is in the same codebase
# from network_analysis_server import network_mcp  # Would need to import
# toolset = FastMCPToolset(network_mcp)

agent = Agent(
    'anthropic:claude-haiku-4-5',
    toolsets=[toolset],
    system_prompt="You are a helpful network analysis assistant."
)

# FastMCPToolset handles all the connection details
async with agent:
    result = await agent.run(
        'Create a path network (1-2-3-4-5) and find the shortest path from 1 to 5'
    )
    print("Agent Response:", result.output)

print("\nFastMCPToolset provides the simplest integration path!")

Agent Response: Perfect! I've created the path network and found the shortest path.

**Network Statistics:**
- **Nodes**: 5
- **Edges**: 4
- **Density**: 0.4

**Shortest Path from 1 to 5:**
- **Path**: 1 → 2 → 3 → 4 → 5
- **Length**: 4 (number of edges/hops)

Since this is a simple linear path network, there's only one way to get from node 1 to node 5, and it requires traversing all 4 edges connecting the nodes in sequence.

FastMCPToolset provides the simplest integration path!


In [14]:
from fastmcp import FastMCP

network_prompts_mcp = FastMCP("NetworkPrompts")

@network_prompts_mcp.prompt()
def analyze_network_structure(graph_id: str) -> str:
    """
    Generate a comprehensive network analysis prompt.

    This creates a structured request for analyzing a network's properties.
    """
    return f"""
Please analyze the network '{graph_id}' and provide:

1. Basic Statistics:
   - Number of nodes and edges
   - Network density
   - Average degree

2. Centrality Analysis:
   - Identify the top 3 nodes by degree centrality
   - Identify the top 3 nodes by betweenness centrality
   - Explain what makes these nodes central

3. Structural Properties:
   - Is the network connected?
   - What is the clustering coefficient?
   - Are there any obvious communities or clusters?

4. Interpretation:
   - What does this structure tell us about the system?
   - What are the implications for information flow?
"""

@network_prompts_mcp.prompt()
def explain_centrality_measures() -> str:
    """
    Educational prompt explaining different centrality measures.
    """
    return """
Please explain the following network centrality measures:

1. Degree Centrality
   - What does it measure?
   - When is it useful?
   - Example interpretation

2. Betweenness Centrality
   - What does it measure?
   - How does it differ from degree?
   - Example interpretation

3. When to use each measure?
   - What questions does each answer?
   - Real-world applications
"""

print("✓ Network prompts server created")

✓ Network prompts server created


### Tools vs Resources vs Prompts

When should you use each?

**Tools** (Functions):
- ✓ When the AI needs to **perform an action**
- ✓ When computation is required
- ✓ When results depend on parameters
- Examples: `calculate_centrality()`, `find_nash_equilibrium()`

**Resources** (Data):
- ✓ When the AI needs to **read information**
- ✓ When data is static or slow-changing
- ✓ When you want to provide datasets
- Examples: adjacency matrices, simulation logs, documentation

**Prompts** (Templates):
- ✓ When you want to **guide the user**
- ✓ When there are common workflows
- ✓ When you want to showcase capabilities
- Examples: "Analyze this network", "Explain this concept"

**A complete MCP server** might have all three:
- **Tools** to perform analysis
- **Resources** to access data
- **Prompts** to guide usage

This makes your server self-documenting and user-friendly.

## Advanced Patterns


### Security Considerations (Preview for A3.02)

Next lecture, we'll dive deep into security.

For now, key considerations:

**Input Validation**:
- MCP servers should validate all inputs
- Use Pydantic models for type safety
- Reject unexpected or malicious data

**Rate Limiting**:
- Limit tool calls per client
- Prevent denial-of-service
- Implement backoff strategies

**Authentication**:
- HTTP servers should require auth tokens
- Verify client identity
- Track usage per user

**Sandboxing**:
- Run untrusted code in isolated environments
- Limit access to file system and network
- Use containers (Docker) for isolation

**Least Privilege**:
- Only expose necessary capabilities
- Separate read and write operations
- Require confirmation for destructive actions

We'll explore these in depth next time.

## Exercises

### Exercise 1: Conceptual Understanding

**Part A**: Explain the difference between:
1. Embedded tools (`@agent.tool` in PydanticAI)
2. MCP servers with tools (`@mcp.tool` in FastMCP)

When would you use each approach?

**Part B**: For each scenario, identify whether you should use a Tool, Resource, or Prompt:
1. Providing access to a dataset of network structures
2. Computing the Nash equilibrium of a game
3. Guiding users through a network analysis workflow
4. Running a simulation for 1000 steps
5. Retrieving historical simulation results

**Part C**: Explain how MCP relates to:
1. Network effects (from Week 3-4)
2. Emergence (from Week 6-7)
3. Strategic interaction (from Week 8-9)

### Exercise 2: Build a Simple MCP Server

Create an MCP server for **statistical analysis** with these tools:

1. `calculate_mean(data: List[float]) -> float`
2. `calculate_std(data: List[float]) -> float`
3. `find_outliers(data: List[float], threshold: float = 2.0) -> List[float]`

**Requirements**:
- Use FastMCP
- Include proper docstrings
- Add type hints
- Handle edge cases (empty lists, etc.)

**Bonus**: Add a resource that returns a summary of all analyses performed.

In [ ]:
# Your code here
from fastmcp import FastMCP, Context
from typing import List, Dict

stats_mcp = FastMCP("Statistics")

# TODO: Implement tools
@stats_mcp.tool()
def calculate_mean(data: List[float]) -> float:
    """Calculate the arithmetic mean of a list of numbers."""
    # TODO: implement
    pass

# TODO: Add other tools

### Exercise 3: Extend the Network Analysis Server

Add these tools to the network analysis MCP server:

1. `calculate_clustering_coefficient(graph_id: str) -> float`
   - Use `nx.average_clustering(G)`
   
2. `find_communities(graph_id: str) -> List[List[int]]`
   - Use `nx.community.greedy_modularity_communities(G)`
   
3. `calculate_diameter(graph_id: str) -> int`
   - Use `nx.diameter(G)` (handle disconnected graphs!)

**Bonus**: Add error handling for disconnected graphs.

In [ ]:
# Your code here
@network_mcp.tool()
def calculate_clustering_coefficient(
    ctx: Context,
    graph_id: str
) -> Dict[str, any]:
    """
    Calculate the average clustering coefficient.

    Clustering coefficient measures how much nodes tend to cluster together.
    Values close to 1 indicate high clustering (friends of friends are friends).
    """
    # TODO: implement
    pass

### Exercise 4: Design an MCP Server for Your Domain

Choose a domain from the course and design (don't implement yet) an MCP server:

**Option A: Blockchain Analysis**
- Tools for querying on-chain data
- Analyzing transaction patterns
- Computing wallet statistics

**Option B: Auction Mechanisms**
- Tools for different auction types
- Computing bidding strategies
- Analyzing revenue and efficiency

**Option C: Your Research Domain**
- What computational tools do you use?
- How could an AI agent access them?
- What would make them useful to others?

**Deliverable**: Write specifications for 5 tools:
- Tool name
- Parameters (with types)
- Return value (with type)
- Docstring explaining what it does
- When/why you'd use it

### Exercise 5: MCP vs Embedded Tools Trade-offs

Consider a project that needs network analysis capabilities.

**Scenario 1**: Single-user research script
- Would you use MCP or embedded tools? Why?
- What are the trade-offs?

**Scenario 2**: Multi-user web application
- Would you use MCP or embedded tools? Why?
- How would you deploy it?

**Scenario 3**: Educational platform for students
- Would you use MCP or embedded tools? Why?
- How would students interact with it?

**Analysis**: For each scenario, discuss:
- Complexity
- Maintenance
- Reusability
- Performance
- Security

## Further Reading

### Official Documentation

**Model Context Protocol**:
- [MCP Specification](https://modelcontextprotocol.io/) - Official protocol documentation
- [Anthropic MCP Announcement](https://www.anthropic.com/news/model-context-protocol) - Vision and motivation
- [MCP Server Examples](https://github.com/modelcontextprotocol/servers) - Community-contributed servers

**FastMCP**:
- [FastMCP Documentation](https://gofastmcp.com/) - Complete guide
- [FastMCP GitHub](https://github.com/jlowin/fastmcp) - Source code and examples
- [FastMCP Examples](https://github.com/jlowin/fastmcp/tree/main/examples) - Sample servers

**PydanticAI**:
- [PydanticAI Docs](https://ai.pydantic.dev/) - Complete framework documentation
- [Agents Guide](https://ai.pydantic.dev/agents/) - Building agents
- [Tools Guide](https://ai.pydantic.dev/tools/) - Tool use patterns

### Related Libraries

**Network Analysis**:
- [NetworkX Documentation](https://networkx.org/documentation/stable/) - Python network analysis
- [NetworkX Tutorial](https://networkx.org/documentation/stable/tutorial.html) - Getting started

**Game Theory**:
- [QuantEcon.game_theory](https://quanteconpy.readthedocs.io/en/latest/game_theory.html) - Game theory tools
- [QuantEcon Lectures](https://quantecon.org/lectures/) - Economic modeling

**Agent-Based Modeling**:
- [Mesa Documentation](https://mesa.readthedocs.io/) - Python ABM framework
- [Mesa Examples](https://github.com/projectmesa/mesa-examples) - Sample models

### Academic Papers

**Tool Use and Function Calling**:
- Schick et al. (2023) "Toolformer: Language Models Can Teach Themselves to Use Tools" [arXiv:2302.04761](https://arxiv.org/abs/2302.04761)
- Qin et al. (2023) "Tool Learning with Foundation Models" [arXiv:2304.08354](https://arxiv.org/abs/2304.08354)
- Patil et al. (2023) "Gorilla: Large Language Model Connected with Massive APIs" [arXiv:2305.15334](https://arxiv.org/abs/2305.15334)

**Multi-Agent Systems**:
- Wang et al. (2024) "A Survey on Large Language Model Based Autonomous Agents" [arXiv:2308.11432](https://arxiv.org/abs/2308.11432)
- Xi et al. (2023) "The Rise and Potential of Large Language Model Based Agents" [arXiv:2309.07864](https://arxiv.org/abs/2309.07864)

### Industry Resources

**MCP Implementations**:
- [Anthropic Claude Desktop](https://www.anthropic.com/claude/desktop) - Native MCP support
- [MCP Server Registry](https://github.com/modelcontextprotocol/servers) - Community servers

**Agent Frameworks**:
- [LangChain](https://python.langchain.com/) - Alternative agent framework
- [AutoGen](https://microsoft.github.io/autogen/) - Multi-agent conversations
- [CrewAI](https://www.crewai.com/) - Role-based agents

### Video Tutorials

**MCP Introductions**:
- Search for "Model Context Protocol tutorial" on YouTube
- Anthropic developer talks on MCP

**FastMCP Walkthroughs**:
- Creator's introduction videos
- Community live coding sessions

### Community

**Discussion and Support**:
- [MCP Discord](https://discord.gg/modelcontextprotocol) - Community discussion
- [PydanticAI Discussions](https://github.com/pydantic/pydantic-ai/discussions) - Q&A
- [FastMCP Issues](https://github.com/jlowin/fastmcp/issues) - Bug reports and features

### Course-Related

**Week 3-5 (Networks)**:
- Newman, M. (2018) "Networks" - Comprehensive network science textbook
- Barabási, A.-L. "Network Science" - Free online textbook

**Week 6-7 (ABMs)**:
- Railsback & Grimm (2019) "Agent-Based and Individual-Based Modeling"
- Wilensky & Rand (2015) "An Introduction to Agent-Based Modeling"

**Week 8-9 (Game Theory)**:
- Osborne & Rubinstein (1994) "A Course in Game Theory"
- Shoham & Leyton-Brown (2008) "Multiagent Systems"

### Next Lecture Preview

**Security Resources** (for L.A3.02):
- [Simon Willison's Blog on Prompt Injection](https://simonwillison.net/2023/Apr/14/worst-that-can-happen/)
- [OWASP Top 10 for LLM Applications](https://owasp.org/www-project-top-10-for-large-language-model-applications/)
- [Claude API reccomendations for security and safeguards](https://support.claude.com/en/articles/9199617-api-safeguards-tools)